In [1]:
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


random.seed(0)
np.random.seed(0)

# Load dataset
X, y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(x_train.shape)
# Train Random Forest
r = RandomForestRegressor()
r.fit(x_train, y_train)
print(r2_score(y_test, r.predict(x_test)))

(353, 10)
0.26865181564422547


In [14]:
from dataclasses import dataclass
import sympy
import numpy as np
import pickle
import copy
from collections import deque
from sympy.utilities import lambdify
from tqdm import tqdm
import warnings

warnings.filterwarnings("error")
rng = np.random.default_rng(0)

STR2SYMPY = {
    'neg': lambda x: -x,
    'abs': sympy.Abs,
    'sqrt': sympy.sqrt,
    'exp': sympy.exp,
    'log': sympy.log,
    'sin': sympy.sin,
    'cos': sympy.cos,
    'tan': sympy.tan,
    'add': lambda x, y : x + y,
    'sub': lambda x, y : x - y,
    'mul': lambda x, y : x * y,
    'div': lambda x, y : x / y,
    'pow': lambda x, y : x ** y
}

binary_functions=np.array(['add','sub','mul','div','pow'])
unary_functions=np.array(['neg','sqrt','exp','log','sin','cos','tan'])
extra_constants=np.array(['2','3','4','5','pi'])
true_variables=['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9']

@dataclass
class ExpNode:
    kind: str
    num_children: int
    children: list
    parent: 'ExpNode' = None

def find_depth(node: ExpNode,currdepth):
    if isinstance(node, np.ndarray):
        return currdepth
    elif node.num_children==1:
        return find_depth(node.children[0],currdepth+1)
    else:
        return max(find_depth(node.children[0],currdepth+1),find_depth(node.children[1],currdepth+1))

def find_variables(root: ExpNode):
    if isinstance(root, np.ndarray):
        return root
    dir=rng.uniform(0,1)
    if dir<1/2 or root.num_children==1:
        return find_variables(root.children[0])
    elif dir>1/2 and root.num_children>1:
        return find_variables(root.children[1])


def visit(formula: ExpNode):
    tovisit=deque()
    tovisit.append((formula,None,0))
    count=0
    while tovisit:
        node,parent,j=tovisit.popleft()
        if not isinstance(node, np.ndarray):
            for i,child in enumerate(node.children):
                tovisit.append((child,node,i))
        count+=1
        if rng.integers(1,count+1)==count:
            item=node
            itemparent=parent
            itemid=j
    return item,itemparent,itemid

def cut_and_shorten(formula: ExpNode):
    cutpoint,cutparent,cutid=visit(formula)
    if cutparent is not None:
        cutparent.children[cutid]=find_variables(cutpoint)
    return formula

def cut_and_regenerate(formula: ExpNode, variables,dummy,simplified):
    cutpoint,cutparent,cutid=visit(formula)
    if cutparent is not None:
        cutparent.children[cutid]=generate_Expression(variables,dummy,simplified)
        cutparent.children[cutid].parent=cutparent
    else:
        formula=generate_Expression(variables,dummy,simplified)
    return formula

def crossover(formula1: ExpNode,formula2: ExpNode):
    cxpoint1,parent1,id1=visit(formula1)
    cxpoint2,parent2,id2=visit(formula2)
    if (parent1 is None) and (not isinstance(cxpoint2,np.ndarray)):
        return cxpoint2
    elif (parent1 is not None) and (not isinstance(cxpoint2,np.ndarray)):
        cxpoint2.parent=parent1
        parent1.children[id1]=cxpoint2
        return formula1
    elif parent1 is not None:
        parent1.children[id1]=cxpoint2
        return formula1
    else:
        return formula1



def choose_function():
    fn_choice=rng.uniform(0,1)
    children=0
    if fn_choice>7/12:
        kind=rng.choice(binary_functions,1)
        children=2
    else:
        kind=rng.choice(unary_functions,1)
        children=1
    return kind[0],children

def generate_subtree(root,depth,max_depth,variables):
    if depth==max_depth:
        for n in range(root.num_children):
            var=rng.choice(variables,1)
            # if var[0]=='const':
            root.children.append(var)
    else:
        for n in range(root.num_children):
            fn,children=choose_function()
            child=ExpNode(fn,children,[],root)
            generate_subtree(child,depth+1,max_depth,variables)
            root.children.append(child)


def generate_Expression(variables: np.ndarray,dummy=True,simplified=False):
    depth=rng.integers(1,3)
    num_variables=rng.integers(1,variables.shape[0]+1)
    #check if replacement mayu be usefull (the same variable is used twice in different fragments of the expression)
    # we may choose just one variable then
    if dummy:
        used_variables=rng.choice(variables,num_variables,replace=False)
    else:
        used_variables=variables
    #print(num_variables,used_variables)
    #using uniform generation
    d=0
    curr_fn,kids=choose_function()
    root=ExpNode(curr_fn,kids,[])
    if simplified:
        generate_subtree(root,1,depth,used_variables)
    else:
        generate_subtree(root,1,depth,np.concatenate([used_variables,extra_constants]))
    return root

        
def parse_Expression(expr: ExpNode):
    if isinstance(expr, np.ndarray):
        return sympy.symbols(expr[0])
    elif expr.kind in STR2SYMPY:
        # Parse children recursively
        parsed_children = [parse_Expression(child) for child in expr.children]
        if expr.kind == 'div' and len(parsed_children) == 2:
            # Check for division by zero
            denominator = parsed_children[1]
            if denominator == 0:
                raise ZeroDivisionError(f"Division by zero detected in expression: {expr}")
        # Apply the corresponding sympy function/operator
        return STR2SYMPY[expr.kind](*parsed_children)
    else:
        raise ValueError(f"Unsupported expression kind: {expr.kind}")
    
class SGA:
    def __init__(self,pop_size,children_size,crossover_prob,mutation_prob,num_iter,variables,data,dummy=False,simplified=False):
        self.pop_size=pop_size
        self.crossover_prob=crossover_prob
        self.mutation_prob=mutation_prob
        self.num_iter=num_iter
        self.variables=variables
        #self.objective_function=objective_function
        self.dummy=dummy
        self.data=data
        self.children_size=children_size
        self.simplified=simplified

    def initial_population(self):
        self.parent_population=[generate_Expression(self.variables,self.dummy,self.simplified) for i in range(self.pop_size)]
        self.best_individual=self.parent_population[0]
        #a litt;e shenanigans
        self.best_objective_value = max([self.objective_function(pop,self.data) for pop in self.parent_population])
    

    def objective_function(self,expression,data):

        try:
            parsed_expression=parse_Expression(expression)
        except:
            return 0
        
        variables=[_ for _ in parsed_expression.atoms() if type(_) is sympy.Symbol and str(_) in true_variables]
        nums=[int(str(_)[1]) for _ in variables]
        #print(parsed_expression,nums)
        # print(parsed_expression)
        # print(variables)
        # print(nums)
        
        #print(inputs)
        #print(x_train[:,nums].shape)
        try:
            fn=lambdify(variables,parsed_expression)
            if len(nums)==0:
                return 0.0
            x_train, x_test, y_train, y_test=data
            #necessary to do it this way because of sympy akwardness
            inputs=tuple([x_train[:,num] for num in nums])
            feature=fn(*inputs).reshape(-1,1)
        #print(feature.shape)
            r = RandomForestRegressor()
            #print(np.hstack((x_train,feature)).shape)
            r.fit(np.hstack((x_train,feature)), y_train)
            test_inputs=tuple([x_test[:,num] for num in nums])
            test_feature=fn(*test_inputs).reshape(-1,1)
            score=r2_score(y_test, r.predict(np.hstack((x_test,test_feature))))
            print(score)
        except RuntimeWarning:
            return 0.0
        except ValueError:
            print('values too large')
            return 0.0
        #score=np.sum(np.sqrt((fn(data[:,nums])-data[:,[data.shape[1]-1]])*(fn(data[:,nums])-data[:,[data.shape[1]-1]])))
        depth=find_depth(expression,0)
        if depth>3*x_train.shape[1]:
            score=score/depth
            #print(score)
        if len(nums)<x_train.shape[1] and self.dummy==False:
            #print((x_train.shape[1]-len(nums))*score)
            return score/(x_train.shape[1]-len(nums))
        else:
            return score

    def fit(self):
        for t in tqdm(range(self.num_iter)):
            self.children_population=[]
            
            for i in range(self.pop_size-1):
                if rng.uniform(0,1)<self.crossover_prob:
                    #check here if copy is deep enough
                    self.children_population.append(crossover(copy.deepcopy(self.parent_population[i]),copy.deepcopy(self.parent_population[i+1])))
                else:
                    self.children_population.append(self.parent_population[i])
            #print(len(self.children_population))

            for i in range(self.children_size-1):
                if rng.uniform(0,1)<self.mutation_prob:
                    self.children_population[i]=cut_and_regenerate(self.children_population[i],self.variables,self.dummy,self.simplified)
                elif rng.uniform(0,1)<self.mutation_prob:
                    self.children_population[i]=cut_and_shorten(self.children_population[i])
                    
            total_pop=self.parent_population+self.children_population
            self.fitness_values=[self.objective_function(pop,self.data) for pop in total_pop]
            print(self.fitness_values)
            self.parent_population=[x for _, x in sorted(zip(self.fitness_values, total_pop), key=lambda pair: pair[0],reverse=True)][:self.pop_size]

            if max(self.fitness_values)>self.best_objective_value:
                self.best_objective_value=max(self.fitness_values)
                self.best_individual=self.parent_population[0]
            
            print(parse_Expression(self.best_individual),self.best_objective_value)
            #print([parse_Expression(pop,)])
            sample=[]
            for i in range(10):
                try:
                    sample.append(parse_Expression(self.parent_population[i]))
                except:
                    print('div by 0')
            print(sample)

        return parse_Expression(self.best_individual)

### 35% accuracy

In [15]:
testing=SGA(100,100,0.95,0.3,10,np.array(['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9']),train_test_split(X, y, test_size=0.2, random_state=0),dummy=True,simplified=True)
testing.initial_population()
testing.fit()

0.27625359759674895
0.24956699131746263
0.2877190164928084
0.2579840325089843
0.26358548165347107
0.26004281108199745
0.24325085120232737
0.26134644143983343
0.2633900216774159
0.266466214682492
0.28122940702111954
0.2370984982782398
0.22312298958770083
0.26921531053359926
0.24825027706470326
0.27293260168136
0.23830652590666634
0.2659112906174581
0.25510514945107277
0.24442060132240873
0.24888577774653864
0.26600276284759283
0.27849244455271527
0.27654411471082885
0.23082560712644118
0.26312324348551197
0.247412314650708
0.2770715192778823
0.24755311929767632
0.25614326020211087
0.2715730731854674
0.2628682027391097
0.25237344700948905
0.24210098975068162
0.27910048204861315
0.2518628479717645
0.25787285654179126
0.26181309955534837
0.278747935795458
0.2578264481880217
0.2988394838116111
0.2632829766477238
0.24335098979007264
0.26636657238570494
0.29434527929645526
0.27386002180051316
0.264308425734601
0.2742124672616858
0.25158112451995185
0.2276633791804732
0.2372243329055298
0.2670

  0%|          | 0/10 [00:00<?, ?it/s]

0.2588696736939573
0.26564441207179723
0.2524957843740915
0.22786326721126515
0.27086700935120167
0.24810579351036544
0.27515984200228916
0.24407931287638507
0.25939594982903047
0.26289895568771815
0.263951195721831
0.24315204504077093
0.22852366570396476
0.2556224031699006
0.25174363756886986
0.28322038677138484
0.2346108686624333
0.2603966693838322
0.26361546244784695
0.23439461464300226
0.23591885476771446
0.275039538225495
0.2691702096271362
0.24243116522327013
0.25860553449909673
0.24269605750953216
0.3059806310651526
0.28414310187696146
0.24479378429580168
0.2809184788731979
0.2567253014736458
0.25363269602797367
0.21479180015859844
0.2267665641086497
0.24689979201990442
0.2779242163841874
0.23895505789630356
0.2619255829698478
0.26950662543813375
0.27212731405614865
0.2559331051932
0.2773599445200946
0.2780011430165362
0.2455234154867595
0.25536013585744965
0.27510959654155975
0.2763185087291914
0.2575442529859072
0.2622871533921526
0.2940403835487002
0.28025214548561983
0.27044

 10%|█         | 1/10 [00:26<04:00, 26.74s/it]

0.25240403014551827
[0.2588696736939573, 0.26564441207179723, 0.2524957843740915, 0.0, 0.22786326721126515, 0.0, 0.27086700935120167, 0.24810579351036544, 0.0, 0.27515984200228916, 0.24407931287638507, 0.25939594982903047, 0.26289895568771815, 0.263951195721831, 0.24315204504077093, 0.22852366570396476, 0.2556224031699006, 0.0, 0.25174363756886986, 0.28322038677138484, 0.0, 0.0, 0.2346108686624333, 0.2603966693838322, 0.26361546244784695, 0.0, 0.23439461464300226, 0.23591885476771446, 0.275039538225495, 0.0, 0.2691702096271362, 0.24243116522327013, 0.25860553449909673, 0.24269605750953216, 0.3059806310651526, 0.0, 0.28414310187696146, 0.24479378429580168, 0.2809184788731979, 0.2567253014736458, 0.25363269602797367, 0.0, 0.21479180015859844, 0.0, 0.2267665641086497, 0.0, 0.24689979201990442, 0.2779242163841874, 0.23895505789630356, 0.0, 0.2619255829698478, 0.0, 0.0, 0.26950662543813375, 0.27212731405614865, 0.2559331051932, 0.2773599445200946, 0.2780011430165362, 0.2455234154867595, 0.2

...

 90%|█████████ | 9/10 [05:20<00:36, 36.42s/it]

0.3039131339856336
[0.32121498275777605, 0.32414706952111016, 0.2809106103060418, 0.33204181980491076, 0.3223949225090853, 0.2939915046957887, 0.24588208660576982, 0.30575818250193076, 0.30465221004271226, 0.29211255479610065, 0.33556204356360475, 0.2603513482687979, 0.2519315243421011, 0.27855565558734474, 0.24850162663340747, 0.2803805468026612, 0.3190890484586879, 0.2898692539300626, 0.23797380565562687, 0.2640240770898644, 0.26129431489394817, 0.2867466966129316, 0.26948115618055024, 0.27204898006096256, 0.25297826244863475, 0.2582458052833808, 0.2618670771679078, 0.27483821040314116, 0.26981267875989934, 0.2819306352504575, 0.26290881051427306, 0.2657196995197487, 0.24437210350701455, 0.28397180480674844, 0.27381492418074516, 0.23310382562199183, 0.306893837323611, 0.29921364402901063, 0.27789941783151506, 0.2744033650858445, 0.29392690075978056, 0.3014762060238352, 0.23780807098818246, 0.25430961038260835, 0.25565327137195026, 0.2579935573513523, 0.25175788298183754, 0.2467794991

100%|██████████| 10/10 [05:58<00:00, 35.88s/it]

0.2714756870953029
[0.309079007290062, 0.3141872422609412, 0.34010740373491655, 0.3097492161293761, 0.3221754413933028, 0.3266735941057125, 0.31086292656439907, 0.2974428077731447, 0.2944862544133424, 0.2787137149453043, 0.2704115319150402, 0.30586323513706004, 0.2839918302016997, 0.3031245007017771, 0.3055428408690565, 0.2580200038529423, 0.30646905630490295, 0.27128105514729706, 0.24584834057350013, 0.2575896691959858, 0.28505124158538353, 0.2675958064281848, 0.24364534510667735, 0.29116629950384454, 0.30341734282396304, 0.25606282206472897, 0.294034797919953, 0.2598074482173187, 0.29827437931809575, 0.25767455225980607, 0.27792655169060476, 0.28754378020897486, 0.29244551256563844, 0.24702440750242938, 0.279150968314536, 0.2763607857072342, 0.27924353654792644, 0.2566763005747903, 0.2832836979406579, 0.2793146352125434, 0.27243088679957395, 0.2541847961645869, 0.2652567292952366, 0.26696832528236647, 0.30771675215192373, 0.2572905115796301, 0.2441300714997744, 0.26488200727209, 0.27

sin(x3 + x7 + x9 + exp(x2))

## Many bugs below

In [27]:
#o to rozbiły się ostatnie 2 godziny - i pewnie powód słabych wyników w poprzednim zadaniu
# głupieje na wieloargumentowych funkcjach i trzeba przekazywać argumenty osobno - co ciekawe pojawiła się w oryginalnych raz funkcja 2 argumentowa
fn=lambdify([sympy.Symbol('x7'), sympy.Symbol('x0')],sympy.parse_expr('cos(x0 - x7)'),'numpy')
print(fn)
input=x_train[0,[7,0]]
print(type(x_train))
print(fn(1,2))
print(input)
fn(input)

<function _lambdifygenerated at 0x7a35aadc8940>
<class 'numpy.ndarray'>
0.5403023058681398
[-0.03949338  0.01264814]


TypeError: _lambdifygenerated() missing 1 required positional argument: 'x0'

In [8]:
testing=SGA(100,100,0.95,0.3,10,np.array(['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9']),train_test_split(X, y, test_size=0.2, random_state=0),dummy=True,simplified=True)
testing.initial_population()
testing.fit()

0.25167928977596565
0.2473011174295534
0.26424749939128445
0.2472395484277905


<lambdifygenerated-1682>:2: RuntimeWarning: invalid value encountered in power
  return cos(x6**x3)
<lambdifygenerated-1682>:2: RuntimeWarning: invalid value encountered in power
  return cos(x6**x3)


0.22993965151430473
0.27277189927276646


<lambdifygenerated-1684>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(sqrt(x0))
<lambdifygenerated-1684>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(sqrt(x0))


0.27508251044910537
0.2772513731492029


...

<lambdifygenerated-2353>:2: RuntimeWarning: invalid value encountered in sqrt
  return tan(tan(sin(sqrt(x0))))
<lambdifygenerated-2353>:2: RuntimeWarning: invalid value encountered in sqrt
  return tan(tan(sin(sqrt(x0))))


0.2664112111828386
0.26166589045432975
0.24983670693832416
0.2635405678695151
0.2787777938865502
0.2741635373554441
0.2604272229396054
0.2890923015605523


<lambdifygenerated-2363>:2: RuntimeWarning: invalid value encountered in power
  return x2**(x8/x2)
/home/mm/projects/ewolucyjne/ewolucyjne/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:839: RuntimeWarning: overflow encountered in cast
  array = numpy.asarray(array, order=order, dtype=dtype)
 20%|██        | 2/10 [01:56<07:46, 58.28s/it]

0.28521743573280134


ValueError: Input X contains infinity or a value too large for dtype('float32').

In [6]:
testing=SGA(100,100,0.95,0.3,10,np.array(['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9']),train_test_split(X, y, test_size=0.2, random_state=0),dummy=True,simplified=True)
testing.initial_population()
testing.fit()

<lambdifygenerated-449>:2: RuntimeWarning: invalid value encountered in power
  return cos(x6**x3)
<lambdifygenerated-449>:2: RuntimeWarning: invalid value encountered in power
  return cos(x6**x3)
<lambdifygenerated-451>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(sqrt(x0))
<lambdifygenerated-451>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(sqrt(x0))
<lambdifygenerated-454>:2: RuntimeWarning: invalid value encountered in log
  return 2*x4*log(x4)
<lambdifygenerated-454>:2: RuntimeWarning: invalid value encountered in log
  return 2*x4*log(x4)
<lambdifygenerated-463>:2: RuntimeWarning: invalid value encountered in power
  return x2**x3
<lambdifygenerated-463>:2: RuntimeWarning: invalid value encountered in power
  return x2**x3
<lambdifygenerated-466>:2: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(sin(x6))
<lambdifygenerated-466>:2: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(sin(x6))
<lambdifygenerated

[0.2635831090978429, 0.2850377006207301, 0.2552795588024219, 0.2713944452514151, 0.2539125850657008, 0.24258539952548486, 0.2639091917585936, 0.2982343326913405, 0.22392094043302146, 0.26313818743074247, 0.25706615103631847, 0.23645597962593712, 0.2676191202711291, 0.2714709980769757, 0.22072707914238687, 0.2448334367382592, 0.2280209275644408, 0.2930839835433824, 0.2769079448455636, 0.24641402746938523, 0.2786177208800662, 0, 0.21912239890827567, 0.2832406694050049, 0.22391104528967354, 0.2742909535404108, 0.2521502633221324, 0.2690486192187712, 0.24576390274730797, 0.2379674684692692, 0.2452740714593985, 0.2851053686330133, 0.26159349924209074, 0.2674641845881821, 0.2821533969261647, 0.25450994299368934, 0.24676775496075554, 0.24122522179775585, 0.23976798599203186, 0.27287427270361586, 0.2413388987207986, 0.2903052633947597, 0.25923623529142403, 0.2742309901987553, 0.23911133476995983, 0, 0.2708044766904384, 0.2581212373483781, 0.28023300070583934, 0.3097414529555761, 0.258139430082

<lambdifygenerated-745>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x8)
<lambdifygenerated-745>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x8)
<lambdifygenerated-751>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + sin(x2))
<lambdifygenerated-751>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + sin(x2))
<lambdifygenerated-753>:2: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(tan(x7))
<lambdifygenerated-753>:2: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(tan(x7))
<lambdifygenerated-754>:2: RuntimeWarning: invalid value encountered in power
  return x2**x3
<lambdifygenerated-754>:2: RuntimeWarning: invalid value encountered in power
  return x2**x3
<lambdifygenerated-757>:2: RuntimeWarning: invalid value encountered in log
  return sqrt(log(x8))
<lambdifygenerated-757>:2: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(log(x8))
<lambdifyge

[0.2974740762946564, 0.27746699962898513, 0.2705589020641185, 0.2838311329422618, 0.2564234194061743, 0.2895259382505083, 0.27119636315001794, 0.2598516102245516, 0.25410797865051515, 0.2782224149429161, 0.2390534619854937, 0.2597318621183403, 0.2518425975477563, 0.25137470735980905, 0.31307133762827655, 0.27233529721694494, 0.27005991535836926, 0.26127584980271223, 0.2490462788998361, 0.27846618122959843, 0.28953799078051257, 0.2830098833727943, 0.26037461500244596, 0.23987983485526598, 0.2600323736339598, 0.24883814192170972, 0.28210568638380096, 0.2516608198642436, 0.22809202228502368, 0.26957729792795415, 0.24483801137959715, 0.2731133192961501, 0.26457574623119495, 0.23031065343683677, 0.27704746088894405, 0.25193154033735055, 0.2578110625109712, 0.281489780758973, 0.27476009332983087, 0.2626872313914588, 0.2930984712953346, 0.25225734888773077, 0.23800086961766753, 0.2997222462899123, 0.25434532996572246, 0.2788036474683462, 0.2600579149797222, 0.2603808299237461, 0.2734570933601

<lambdifygenerated-944>:2: RuntimeWarning: invalid value encountered in power
  return x0**x3
<lambdifygenerated-944>:2: RuntimeWarning: invalid value encountered in power
  return x0**x3
<lambdifygenerated-945>:2: RuntimeWarning: invalid value encountered in power
  return x9**((1/2)*x6 + (1/2)*x9)
<lambdifygenerated-945>:2: RuntimeWarning: invalid value encountered in power
  return x9**((1/2)*x6 + (1/2)*x9)
<lambdifygenerated-946>:2: RuntimeWarning: invalid value encountered in sqrt
  return x0 + sqrt(x8)
<lambdifygenerated-946>:2: RuntimeWarning: invalid value encountered in sqrt
  return x0 + sqrt(x8)
<lambdifygenerated-947>:2: RuntimeWarning: invalid value encountered in power
  return x7**x2
<lambdifygenerated-947>:2: RuntimeWarning: invalid value encountered in power
  return x7**x2
<lambdifygenerated-948>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x8)
<lambdifygenerated-948>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x8)
<l

...

[0.3018858088655092, 0.2889545233024221, 0.3125967973605992, 0.32871552135477833, 0.25081601214083016, 0.27001372129711954, 0.27372976284340744, 0.3028768782453036, 0.2504951813997186, 0.26991806006453756, 0.2627358698774026, 0.3110389290864144, 0.2732825694126615, 0.2626942638233841, 0.2552809606874339, 0.2743045830267189, 0.2745154689966177, 0.26606400537097696, 0.2588869255564794, 0.27624615520438944, 0.2619647592797436, 0.3201483130366577, 0.2610444756711937, 0.26605812109120264, 0.28203477374612607, 0.2553917950574508, 0.24814894321551073, 0.29181584773960334, 0.30802250945201826, 0.27627213937753703, 0.2760156921447494, 0.24156246015897764, 0.2833829028878848, 0.26951594584806715, 0.2709127023010067, 0.2862740790618814, 0.2596217352642195, 0.25134227622468786, 0.21141692194480777, 0.2703399029969491, 0.29768167929438727, 0.2798644053014515, 0.27190808294830593, 0.2641049514812409, 0.25805358401666656, 0.26580394759407766, 0.28998399771433137, 0.2593949289815358, 0.270954983880422

<lambdifygenerated-1342>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(tan(x0 + x2 + sqrt(x8)))
<lambdifygenerated-1342>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(tan(x0 + x2 + sqrt(x8)))
<lambdifygenerated-1346>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x8)
<lambdifygenerated-1346>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x8)
<lambdifygenerated-1347>:2: RuntimeWarning: invalid value encountered in power
  return x8**x2
<lambdifygenerated-1347>:2: RuntimeWarning: invalid value encountered in power
  return x8**x2
<lambdifygenerated-1348>:2: RuntimeWarning: invalid value encountered in power
  return x5 + 2*x8 + (-x5)**(-x3)
<lambdifygenerated-1348>:2: RuntimeWarning: invalid value encountered in power
  return x5 + 2*x8 + (-x5)**(-x3)
<lambdifygenerated-1349>:2: RuntimeWarning: invalid value encountered in sqrt
  return exp(sqrt(x7))
<lambdifygenerated-1349>:2: RuntimeWarning: invalid v

[0.35939640084730284, 0.2834472408207038, 0.2976205044769219, 0.2687987565509249, 0.2645941357284437, 0.3187555220908995, 0.2914162772060188, 0.2665135452829034, 0.2858390146315789, 0.2898449337010822, 0.3070597196732213, 0.3095363526645989, 0.3185505896404849, 0.25065043435829815, 0.27074765236170584, 0.24459646974726412, 0.30177962648372136, 0.26418581338317626, 0.3141895475488763, 0.28100571280045883, 0.2808428645086417, 0.2926711349807515, 0.2676134395473372, 0.31015748523018327, 0.24259604469265228, 0.2767108603656825, 0.2998134203072458, 0.2663415144032335, 0.26511253847587457, 0.273957627880136, 0.27407924918343485, 0.24526193500911286, 0.29267156290845175, 0.2475589185616045, 0.243028421044598, 0.27297699507192696, 0.2791826415377696, 0.2656029127258127, 0.26286626731392826, 0.2617832662240257, 0.26669850908863524, 0.24944341815550686, 0.2830246774447288, 0.2650492314697487, 0.27006414358204467, 0.2665335811952787, 0.27044324041546786, 0.270206715156293, 0.2864083712320873, 0.2

<lambdifygenerated-1541>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(tan(x0 + x2 + sqrt(x8)))
<lambdifygenerated-1541>:2: RuntimeWarning: invalid value encountered in sqrt
  return sin(tan(x0 + x2 + sqrt(x8)))
<lambdifygenerated-1542>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x7)
<lambdifygenerated-1542>:2: RuntimeWarning: invalid value encountered in log
  return log(x2 + x7)
<lambdifygenerated-1543>:2: RuntimeWarning: invalid value encountered in power
  return x8**x2
<lambdifygenerated-1543>:2: RuntimeWarning: invalid value encountered in power
  return x8**x2
<lambdifygenerated-1545>:2: RuntimeWarning: invalid value encountered in log
  return -log(x2 + x8) - tan(tan(x1*sin(x6)))
<lambdifygenerated-1545>:2: RuntimeWarning: invalid value encountered in log
  return -log(x2 + x8) - tan(tan(x1*sin(x6)))
<lambdifygenerated-1546>:2: RuntimeWarning: invalid value encountered in sqrt
  return x0 + sqrt(x8)
<lambdifygenerated-1546>:2: Runti

ValueError: Input X contains infinity or a value too large for dtype('float32').

In [29]:
#o to rozbiły się ostatnie 2 godziny - i pewnie powód słabych wyników w poprzednim zadaniu
fn=lambdify([sympy.Symbol('x7'), sympy.Symbol('x0')],sympy.parse_expr('cos(x0 - x7)'),'numpy')
print(fn)
input7=x_train[:,7]
input0=x_train[:,0]
print(type(x_train))
print(fn(input7,input0))
# print(input)
# fn(input)

<function _lambdifygenerated at 0x7a35aadcbac0>
<class 'numpy.ndarray'>
[0.99864094 0.99952476 0.99955689 0.99942278 0.99756995 0.9999682
 0.99823595 0.99999388 0.99703746 0.99931426 0.99901888 0.99968014
 0.99986714 0.99983261 0.99698672 0.9996733  0.99895135 0.99623998
 0.99320032 0.99894508 0.99917316 0.99901888 0.99995381 0.99621385
 0.99594345 0.99640372 0.99984868 0.99989248 0.99432045 0.99867078
 0.99901888 0.99997852 0.99432045 0.99999725 0.99905344 0.99964321
 0.99976542 0.9998375  0.99968738 0.99936585 0.9999479  0.99945112
 0.99303395 0.99705812 0.99905696 0.99992578 0.99896363 0.99918407
 0.99993263 0.99838535 0.99946123 0.9994886  0.986409   0.9998375
 0.99908078 0.99905594 0.99998213 0.99999827 0.99992578 0.9987354
 0.999242   0.99981088 0.99936585 0.99999725 0.99965842 0.99132711
 0.99938618 0.99914959 0.99993263 0.99677311 0.99873829 0.99703746
 0.99901888 0.99844503 0.99887884 0.9991053  0.99699763 0.99933541
 0.9978719  0.99882367 0.99801369 0.99978969 0.99999582 0.99